In [ ]:
from model import BiLSTM_crf
from utils import load_data_and_labels,save_pred,transformer_x,transformer_y
from sklearn_crfsuite import metrics

x_train, y_train = load_data_and_labels('../data/train.txt')
transformer_x = transformer_x()
x_train = transformer_x.fit(x_train)
vocab_size = transformer_x.vocab_size
transformer_y = transformer_y(transformer_x.max_len)
y_train = transformer_y.to_onehot(y_train)

use_crf = True
model = BiLSTM_crf(num_labels=8, word_vocab_size = vocab_size,max_seq_len=transformer_x.max_len,use_crf = use_crf)
model = model.build()
model.summary()


# model.fit(x_train,y_train,verbose = 1,batch_size = 12,epochs= 1)
x_test, y_test = load_data_and_labels('../data/dev.txt')
x_test = transformer_x.tran(x_test)
y_test_onehot = transformer_y.to_onehot(y_test)
model.fit(x_train,y_train,validation_data = (x_test,y_test_onehot),verbose = 1,epochs= 100)

pred = model.predict(x_test)
pred = transformer_y.to_tag(pred)
print(metrics.flat_f1_score(y_test, pred,
                      average='weighted', labels=transformer_y.tags))

# group B and I results
labels = transformer_y.tags
sorted_labels = sorted(
    labels,
    key=lambda name: (name[1:], name[0])
)
print(metrics.flat_classification_report(
    y_test, pred, labels=sorted_labels, digits=3
))
save_pred(pred)

/home/k/anaconda3/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


_________________________________________________________________
Layer (type)                 Output Shape              Param #   
word_input (InputLayer)      (None, 100)               0         
_________________________________________________________________
word_embedding (Embedding)   (None, 100, 100)          519800    
_________________________________________________________________
dropout (Dropout)            (None, 100, 100)          0         
_________________________________________________________________
BiLSTM (Bidirectional)       (None, 100, 120)          212160    
_________________________________________________________________
time_distributed_1 (TimeDist (None, 100, 100)          12100     
_________________________________________________________________
crf (CRF)                    (None, 100, 8)            888       
Total params: 744,948
Trainable params: 744,948
Non-trainable params: 0
_________________________________________________________________
Trai

298770/298770 [==============================] - 3956s 13ms/step - loss: 21.3702 - acc: 0.9828 - val_loss: 21.8531 - val_acc: 0.9774
Epoch 50/100
298770/298770 [==============================] - 3962s 13ms/step - loss: 21.3701 - acc: 0.9826 - val_loss: 21.8522 - val_acc: 0.9786
Epoch 51/100
298770/298770 [==============================] - 3956s 13ms/step - loss: 21.3703 - acc: 0.9824 - val_loss: 21.8533 - val_acc: 0.9773
Epoch 52/100
298770/298770 [==============================] - 3959s 13ms/step - loss: 21.3702 - acc: 0.9825 - val_loss: 21.8519 - val_acc: 0.9780
Epoch 53/100
298770/298770 [==============================] - 3958s 13ms/step - loss: 21.3702 - acc: 0.9826 - val_loss: 21.8523 - val_acc: 0.9773
Epoch 54/100
298770/298770 [==============================] - 3960s 13ms/step - loss: 21.3702 - acc: 0.9825 - val_loss: 21.8530 - val_acc: 0.9773
Epoch 55/100
298770/298770 [==============================] - 3993s 13ms/step - loss: 21.3703 - acc: 0.9826 - val_loss: 21.8518 - val_acc

In [ ]:
labels.remove('O')
sorted_labels = sorted(
    labels,
    key=lambda name: (name[1:], name[0])
)
print(metrics.flat_classification_report(
    y_test, pred, labels=sorted_labels, digits=3
))

只跑一个epoch，模型将所有tag都预测为‘O’，这是可以理解的，在没有得到足够的训练的情况下，这种保守的预测可以得到较高的crf.accuracy；在之后的训练中，想得到更高的accuracy就必须改变保守的策略，从而会预测出entity。

In [ ]:
import os
os.mkdir('../data/100epochs')
model.save('../data/100epochs/100epochs_model.h5')

In [ ]:
transformer_x.vocab_size

In [ ]:
transformer_x.max_len

Epoch 44/100
298770/298770 [==============================] - 3959s 13ms/step - loss: 21.3703 - acc: 0.9827 - val_loss: 21.8520 - val_acc: 0.9804

In [ ]:
from seqeval.metrics import classification_report
from functools import reduce
y_test = reduce(lambda x,y : x + y, y_test)
pred = reduce(lambda x,y : x + y, pred)
print(classification_report(y_test,pred,digits=4))